Running a simulation with SHyFT
=========

### This notebook is guiding through the simulation process of a catchment. The following steps are described:
1. **Loading required python modules and setting path to SHyFT installation**
2. **Configuration of a SHyFT calibration**
3. **Running a SHyFT calibration**
4. **Inspecting the calibration results**

### 1. Loading required python modules and setting path to SHyFT installation

In [1]:
# importing thrid-party python modules
import sys
import os

In [2]:
# setting the path to the shyft build
shyft_path = os.path.abspath("../../../shyft")
sys.path.insert(0,shyft_path)

In [7]:
# importing the shyft modules needed for running a calibration
from shyft.repository.default_state_repository import DefaultStateRepository
from shyft.orchestration.configuration.yaml_configs import YAMLCalibConfig
from shyft.orchestration.simulators.config_simulator import ConfigCalibrator

###2. Configuration of a SHyFT calibration

In [8]:
# set up configuration using *.yaml configuration files
config_file_path = os.path.abspath("./sagelva-config/sagelva_hbv/sagelva_calibration.yaml") # here is the *.yaml file
cfg = YAMLCalibConfig(config_file_path, "sagelva")

In [9]:
# initialize an instance of the orchestration's ConfigCalcalibrator class, which has all the functionality needed
# to run a calibration using the above initiated configuration
calib = ConfigCalibrator(cfg)
n_cells = calib.region_model.size()
state_repos = DefaultStateRepository(calib.region_model.__class__, n_cells)
#calib.init()

###3. Running a SHyFT calibration

In [10]:
# once the calibrator is set up, all you need to do is running the calibration...
# the calibrated parameters are stored in a model.yaml. 
results = calib.calibrate(cfg.sim_config.time_axis, state_repos.get_state(0), 
                          cfg.optimization_method['name'],
                          cfg.optimization_method['params'])

Creating TargetSpecificationVector...
Fetching sources and running interpolation...
Calibrating...


KeyError: 'soil'

###4. Inspecting the calibration results
First the Nash-Suttcliffe-efficiency of the calibrated simulation is computed to see the quality of the calibration.
Then the calibrated model parameters are accessed and printed out.

In [ ]:
# Get NSE of calibrated run:
result_params = []
for i in range(results.size()):
    result_params.append(results.get(i))
print("Final NSE =", 1-calib.optimizer.calculate_goal_function(result_params))

In [ ]:
# Check out parameters. Print a warning if parameter interval is reached.
diff = 1.0E-3
print("{0:30s} {1:10s} {2:10s} {3:10s}".format("PARAM-NAME", "CALIB-PARAM", "MIN", "MAX"))
for i in range(results.size()): 
    p_cal = results.get(i) # calibrated parameter
    p_min = calib.p_min.get(i) # min parameter set in the calibration.yaml file
    p_max = calib.p_max.get(i) # max parameter set in the calibration.yaml file
    print("{0:30s} {1:10f} {2:10f} {3:10f}".format(results.get_name(i),p_cal, p_min, p_max,))